In [15]:
import pandas as pd
import re
from konlpy.tag import Okt

# 데이터 로드
file_path = r"C:\Users\USER\Desktop\24-2\TA\Team7_final_ta\TA Team7 Team Project (1)\data\Model 2_dataset.xlsx"
df = pd.read_excel(file_path, sheet_name='Sheet1')

# 간단한 텍스트 전처리 함수
def simple_preprocess(text):
    if pd.isnull(text):  # 결측값 처리
        return ""
    # 1. 영어 제거
    text = re.sub(r'[a-zA-Z]', '', text)
    # 2. 특수 문자 제거 (한글 및 공백 제외)
    text = re.sub(r'[^가-힣\s]', '', text)
    # 3. 공백 정리
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# '리뷰 (Longest)' 컬럼에 전처리 적용
df['리뷰'] = df['리뷰'].apply(simple_preprocess)


In [16]:
df.head()

,가게명,주소,가게분류,동반자 1순위,동반자 2순위,동반자 1순위 비율,동반자 2순위 비율,대기시간 1순위,대기시간 2순위,대기시간 1순위 비율,대기시간 2순위 비율,리뷰,요약주소
0,Just Steak,서울 강남구 언주로152길 11-7,양식,연인・배우자,친구,0.625000,0.125000,대기 시간 바로 입장,대기 시간 10분 이내,0.958333,0.041667,어서 많이 없어서 딱 스테이크 전문점이란 느낌이 들었습니다스테이크 너무 주변사람들에...,압구정
1,Lu307 이자카야,서울 강남구 압구정로10길 30-7 1층,일식,친구,연인・배우자,0.531915,0.244681,대기 시간 바로 입장,대기 시간 30분 이내,0.925532,0.042553,너어무 시끄러워요 분위기도 좋고 맛있고 친절하셨어요 일차만 먹었는데도 배도 엄청 불...,압구정
2,갓잇 압구정 도산공원점,서울 강남구 언주로164길 24 아크로스빌딩 지상1층 GODEAT,양식,NaN,NaN,NaN,NaN,대기 시간 바로 입장,NaN,NaN,NaN,도산공원점에서 처음 먹어보고 다른 지점으로도 몇 번 갔었는데 갈 때마다 맛있어요 도...,압구정
3,갓포 모로이 도산공원점,서울 강남구 언주로172길 30 1층 갓포 모로미,일식,연인・배우자,친구,0.337500,0.312500,대기 시간 바로 입장,대기 시간 10분 이내,0.937500,0.050000,맛고 사람들이 친절해요 연말 모임으로 근사하게 잘 보냈습니다코스 중에 회는 정말 최...,압구정
4,강남철판요리,서울 강남구 압구정로28길 9-2 1층 강남철판요리,일식,연인・배우자,친구,0.545455,0.181818,대기 시간 바로 입장,NaN,1.000000,NaN,잘 서울 왔다에 방문했는데 요리 해주셨어요 매장도 깔끔하고 좋았어요 아이가 쉐프님의...,압구정


## Input에 대해 키워드 추출

In [17]:
import re
import pandas as pd
import numpy as np

def normalize_keywords(extracted):
    """
    Normalize extracted keywords to handle synonyms and equivalences.
    """
    if extracted["주소"] == "건대":
        extracted["주소"] = "건대입구"
    if extracted["동반자"] in ["애인", "배우자"]:
        extracted["동반자"] = "애인"
    if extracted["대기시간"]:
        if extracted["대기시간"] in ["대기시간없음", "없음", "바로 입장", "즉시 입장"]:
            extracted["대기시간"] = "대기 시간 바로 입장"
        elif "10분" in extracted["대기시간"] or "10" in extracted["대기시간"]:
            extracted["대기시간"] = "대기 시간 10분 이내"
        elif "30분" in extracted["대기시간"] or "30" in extracted["대기시간"]:
            extracted["대기시간"] = "대기 시간 30분 이내"
    return extracted

def extract_keywords(sentence: str):
    """
    Extract specific keywords from a sentence.
    Keywords: 동반자, 대기시간, 가게분류, 주소.
    """
    patterns = {
        "동반자": r"(가족|친구|연인|혼자|아이|부모님|애인|배우자)",
        "대기시간": r"(10\s*분|30\s*분|즉시\s*입장|없음|바로\s*입장|10|30|대기시간없음|대기시간 없으면|대기시간 없는)",
        "가게분류": r"(한식|중식|일식|양식|아시안)",
        "주소": r"(성수|건대|왕십리|압구정)"
    }

    extracted = {key: re.search(pattern, sentence) for key, pattern in patterns.items()}
    extracted = {key: match.group(0) if match else None for key, match in extracted.items()}
    
    return normalize_keywords(extracted)

def ask_question_for_keyword(keyword):
    """
    Generate a question for a specific missing keyword.
    """
    questions = {
        "동반자": "누구랑 같이 가실 건가요? (예: 가족, 친구, 연인, 혼자)",
        "가게분류": "어떤 종류의 음식을 먹고싶으세요? (예: 일식, 한식, 중식, 양식, 아시안)",
        "주소": "어느 지역에서 찾고 계신가요? (예: 성수, 건대, 왕십리, 압구정)",
        "대기시간": "대기시간은 어느정도가 괜찮으실까요? (예: 10분, 30분, 즉시 입장, 없음, 대기시간 없음)"
    }
    return questions.get(keyword, "")

def process_input():
    """
    Process user input and return collected information as a dictionary.
    """
    collected_info = {
        "동반자": None,
        "대기시간": None,
        "가게분류": None,
        "주소": None
    }

    input_log = []

    while True:
        # Check if all required information is filled
        if all(collected_info.values()):
            break

        # Ask for user input
        user_input = input("계획을 말씀해주세요: ").strip()
        input_log.append(user_input)  # Save input to log
        keywords = extract_keywords(user_input)

        # Update collected information
        for key, value in keywords.items():
            if value:
                collected_info[key] = value

        # Ask for missing information
        for key in collected_info:
            if collected_info[key] is None:
                question = ask_question_for_keyword(key)
                if question:
                    print(question)
                    additional_input = input().strip()
                    input_log.append(additional_input)  # Save input to log
                    if additional_input in ["상관없음", "상관없어", "몰라", "아무거나", ""]:
                        collected_info[key] = np.nan
                    else:
                        new_info = extract_keywords(additional_input)
                        if new_info.get(key):
                            collected_info[key] = new_info[key]

    # Combine input log into a single string
    combined_text = " ".join(input_log)

    return collected_info, combined_text, input_log

def main():
    # Process input and collect information
    collected_info, combined_text, input_log = process_input()
    
    # Convert to DataFrame
    df_input_info = pd.DataFrame([collected_info])
    print("\n네. 요구사항에 맞추어 음식점 세곳을 추천해드릴게요.")
    print(df_input_info)

    # Print combined text log
    print("\n저장된 전체 입력:")
    print(combined_text)
    
    # Save DataFrame to file
    df_input_info.to_csv("plans.csv", index=False)
    
    # Save combined text log to file
    with open("combined_input_log.txt", "w", encoding="utf-8") as file:
        file.write(combined_text)
    
    # Save input log as a list to a file
    with open("combined_input_list.txt", "w", encoding="utf-8") as file:
        for item in input_log:
            file.write(f"{item}\n")

    return df_input_info




# 추천모델

### TFIDF 유사도 검사

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

def calculate_similarity_tfidf(data, combined_all_input):
    """
    Calculate cosine similarity using TF-IDF.
    """
    vectorizer = TfidfVectorizer()
    # Combine input with review data
    tfidf_matrix = vectorizer.fit_transform([combined_all_input] + data['리뷰'].tolist())
    similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
    data['유사도 점수'] = similarities
    return data


### SBERT 유사도 검사

In [19]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

def calculate_similarity_sbert(data, combined_all_input):
    """
    Calculate cosine similarity using Sentence-BERT.
    """
    model = SentenceTransformer('all-MiniLM-L6-v2')  # Pretrained Sentence-BERT model
    # Generate embeddings for input and reviews
    sentences = [combined_all_input] + data['리뷰'].tolist()
    embeddings = model.encode(sentences)
    similarities = cosine_similarity([embeddings[0]], embeddings[1:]).flatten()
    data['유사도 점수'] = similarities
    return data


### 점수 부여

In [20]:

# 기존의 대기시간 평가 함수
def evaluate_wait_time(row, input_wait_time):
    wait_time_order = {"대기 시간 바로 입장": 1, "대기 시간 10분 이내": 2, "대기 시간 30분 이내": 3}
    if pd.isnull(row['대기시간 1순위']):
        return 0
    row_wait_time_rank = wait_time_order.get(row['대기시간 1순위'], float('inf'))
    input_wait_time_rank = wait_time_order.get(input_wait_time, float('inf'))
    if row_wait_time_rank < input_wait_time_rank:
        return 0.5 * row.get('대기시간 1순위 비율', 0)
    elif row_wait_time_rank == input_wait_time_rank:
        return 0.4 * row.get('대기시간 1순위 비율', 0)
    else:
        return 0.2 * row.get('대기시간 1순위 비율', 0)


# 점수 계산 함수 (유사도 점수 포함)
def calculate_score(row, input_data):
    score = 0

    # 동반자 조건 점수
    if row['동반자 1순위'] == input_data.iloc[0]['동반자']:
        score += 0.5 * row['동반자 1순위 비율']
    else:
        score += 0.3 * row['동반자 1순위 비율']

    # 대기시간 조건 점수
    score += evaluate_wait_time(row, input_data.iloc[0]['대기시간'])

    # 유사도 점수 추가
    score += 0.5* row['유사도 점수']

    return score

In [21]:
# 추천 함수 (유사도 점수 추가)
def recommend_restaurants(input_data, combined_all_input, top_n=3):
    # 유사도 점수 계산
    global df
    df = calculate_similarity_sbert(df, combined_all_input)

    # 가게분류와 주소는 완전히 일치해야 함
    filtered = df[
        (df['가게분류'] == input_data.iloc[0]['가게분류']) &
        (df['요약주소'] == input_data.iloc[0]['주소'])
    ]

    # 점수 계산
    filtered['점수'] = filtered.apply(lambda row: calculate_score(row, input_data), axis=1)

    # 점수 기준으로 상위 N개 음식점 선택
    top_recommendations = filtered.nlargest(top_n, '점수')

    # 결과 데이터 정리
    recommendations = []
    for _, row in top_recommendations.iterrows():
        recommendations.append({
            "가게명": row['가게명'],
            "가게분류": row['가게분류'],
            "주소": row['주소'],
            "동반자 정보": f"이 가게의 방문자는 {row['동반자 1순위 비율']*100:.0f}%가 '{row['동반자 1순위']}'였습니다.",
            "대기시간 정보": f"대기시간은 {row['대기시간 1순위 비율']*100:.0f}%가 '{row['대기시간 1순위']}'로 나타났습니다.",
            "유사도 점수": f"{row['유사도 점수']:.2f}",
            "점수": row['점수']
        })

    return recommendations

### 입력받은 input을 하나의 문장으로 합쳐 저장

In [22]:
def load_combined_input_list(filename="combined_input_list.txt"):
    """
    Load combined input list from a file and return it as a single string in a list.
    """
    # 파일에서 각 줄을 읽어서 리스트로 저장
    with open(filename, "r", encoding="utf-8") as file:
        input_list = file.readlines()

    # 각 줄의 줄바꿈(\n)을 제거하고 하나의 문장으로 결합
    combined_text = " ".join(line.strip() for line in input_list)

    # 한 문장을 리스트로 반환
    return [combined_text]




### 각 카테고리별 질문을 통해, input된 내용을 데이터 프레임에 저장, 아직 받지 못한 정보는 다시 물어보기

In [23]:
if __name__ == "__main__":
    df2 = main()


Input_data = pd.read_csv("plans.csv")

대기시간은 어느정도가 괜찮으실까요? (예: 10분, 30분, 즉시 입장, 없음, 대기시간 없음)
어떤 종류의 음식을 먹고싶으세요? (예: 일식, 한식, 중식, 양식, 아시안)
어느 지역에서 찾고 계신가요? (예: 성수, 건대, 왕십리, 압구정)

네. 요구사항에 맞추어 음식점 세곳을 추천해드릴게요.
  동반자         대기시간 가게분류  주소
0  가족  대기 시간 바로 입장   양식  성수

저장된 전체 입력:
가족과 식사를 하고싶어. 좀 깔끔하고 분위기 좋은 식당 추천해줘. 즉시 입장이 좋겠지? 양식을 먹고싶어. 이븐하게 익은 스테이크와, 간이 타이트하고 면의 익힘정도도 적당한 파스타를 잘하는 집을 추천해줘봐. 성수에서 먹을거야~


In [24]:
Input_data

,동반자,대기시간,가게분류,주소
0,가족,대기 시간 바로 입장,양식,성수


In [25]:
def load_combined_input_list(filename="combined_input_list.txt"):
    """
    Load combined input list from a file and return it as a single combined string.
    """
    # 파일에서 내용을 읽고 줄바꿈을 제거한 후 결합
    with open(filename, "r", encoding="utf-8") as file:
        combined_text = " ".join(file.read().splitlines())
    
    return combined_text  # 리스트가 아니라 문자열로 반환

# 호출 예시
combined_all_input = load_combined_input_list()
print(combined_all_input)



가족과 식사를 하고싶어. 좀 깔끔하고 분위기 좋은 식당 추천해줘. 즉시 입장이 좋겠지? 양식을 먹고싶어. 이븐하게 익은 스테이크와, 간이 타이트하고 면의 익힘정도도 적당한 파스타를 잘하는 집을 추천해줘봐. 성수에서 먹을거야~


### 각 카테고리에 관한 input된 dataframe

In [26]:
Input_data

,동반자,대기시간,가게분류,주소
0,가족,대기 시간 바로 입장,양식,성수


### 추천 결과

In [27]:
# 호출 예시
recommendations = recommend_restaurants(Input_data, combined_all_input)

# 결과 출력
if recommendations:
    print("추천 음식점 리스트:")
    for i, rec in enumerate(recommendations, start=1):
        print(f"\n{i}. {rec['가게명']} ({rec['가게분류']})")
        print(f"   주소: {rec['주소']}")
        print(f"   {rec['동반자 정보']}")
        print(f"   {rec['대기시간 정보']}")
        print(f"   유사도 점수: {rec['유사도 점수']}")
        print(f"   최종 점수: {rec['점수']:.3f}")
else:
    print("조건에 맞는 음식점이 없습니다.")


추천 음식점 리스트:

1. 엘로코 성수 (양식)
   주소: 서울 성동구 동일로 75-18 1F, 2F
   이 가게의 방문자는 51%가 '연인・배우자'였습니다.
   대기시간은 97%가 '대기 시간 바로 입장'로 나타났습니다.
   유사도 점수: 0.77
   최종 점수: 0.930

2. 리틀포레스트 (양식)
   주소: 서울 성동구 성수일로12길 23 2층
   이 가게의 방문자는 57%가 '친구'였습니다.
   대기시간은 100%가 '대기 시간 바로 입장'로 나타났습니다.
   유사도 점수: 0.70
   최종 점수: 0.922

3. 리타르단도 (양식)
   주소: 서울 성동구 뚝섬로 393 B1 리타르단도
   이 가게의 방문자는 51%가 '친구'였습니다.
   대기시간은 99%가 '대기 시간 바로 입장'로 나타났습니다.
   유사도 점수: 0.74
   최종 점수: 0.920


C:\Users\USER\AppData\Local\Temp\ipykernel_30212\165014587.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['점수'] = filtered.apply(lambda row: calculate_score(row, input_data), axis=1)


In [28]:
df[['가게명', '유사도 점수']]

,가게명,유사도 점수
0,Just Steak,0.734839
1,Lu307 이자카야,0.764099
2,갓잇 압구정 도산공원점,0.697799
3,갓포 모로이 도산공원점,0.764161
4,강남철판요리,0.706861
...,...,...
181,타이거풀,0.693363
182,티엔티엔,0.728019
183,해피베네핏 성수,0.667761
184,홍성집,0.692616
